In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving out_pol - Copy (2).txt to out_pol - Copy (2).txt
User uploaded file "out_pol - Copy (2).txt" with length 2173397 bytes


In [0]:
import numpy as np

In [0]:
from __future__ import print_function
import numpy as np

# method for generating text
def generate_text(model, length, vocab_size, ix_to_char):
	# starting with random character
	ix = [np.random.randint(vocab_size)]
	y_char = [ix_to_char[ix[-1]]]
	X = np.zeros((1, length, vocab_size))
	for i in range(length):
		# appending the last predicted character to sequence
		X[0, i, :][ix[-1]] = 1
		print(ix_to_char[ix[-1]], end="")
		ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
		y_char.append(ix_to_char[ix[-1]])
	return ('').join(y_char)

# method for preparing the training data
def load_data(data_dir, seq_length):
	data = open(data_dir, 'r').read()
	chars = list(set(data))
	VOCAB_SIZE = len(chars)

	print('Data length: {} characters'.format(len(data)))
	print('Vocabulary size: {} characters'.format(VOCAB_SIZE))

	ix_to_char = {ix:char for ix, char in enumerate(chars)}
	char_to_ix = {char:ix for ix, char in enumerate(chars)}

	X = np.zeros((len(data)/seq_length, seq_length, VOCAB_SIZE))
	y = np.zeros((len(data)/seq_length, seq_length, VOCAB_SIZE))
	for i in range(0, len(data)/seq_length):
		X_sequence = data[i*seq_length:(i+1)*seq_length]
		X_sequence_ix = [char_to_ix[value] for value in X_sequence]
		input_sequence = np.zeros((seq_length, VOCAB_SIZE))
		for j in range(seq_length):
			input_sequence[j][X_sequence_ix[j]] = 1.
			X[i] = input_sequence

		y_sequence = data[i*seq_length+1:(i+1)*seq_length+1]
		y_sequence_ix = [char_to_ix[value] for value in y_sequence]
		target_sequence = np.zeros((seq_length, VOCAB_SIZE))
		for j in range(seq_length):
			target_sequence[j][y_sequence_ix[j]] = 1.
			y[i] = target_sequence
	return X, y, VOCAB_SIZE, ix_to_char


In [8]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
import argparse
import pandas as pd 
from keras.models import Sequential


Using TensorFlow backend.


In [9]:
ap = argparse.ArgumentParser()
ap.add_argument('--batch_size', default=100, type=int, help='batch size')
ap.add_argument('--train_steps', default=1000, type=int,
                    help='number of training steps')

_StoreAction(option_strings=['--train_steps'], dest='train_steps', nargs=None, const=None, default=1000, type=<type 'int'>, choices=None, help='number of training steps', metavar=None)

In [0]:
import sys; sys.argv=['']; del sys

In [0]:
ap = argparse.ArgumentParser()
ap.add_argument('--batch_size', default=100, type=int, help='batch size')
ap.add_argument('--train_steps', default=1000, type=int,
                    help='number of training steps')
ap.add_argument('-data_dir', default='out_pol - Copy (2).txt')
ap.add_argument('-batch_size', type=int, default=50)
ap.add_argument('-layer_num', type=int, default=2)
ap.add_argument('-seq_length', type=int, default=50)
ap.add_argument('-hidden_dim', type=int, default=500)
ap.add_argument('-generate_length', type=int, default=500)
ap.add_argument('-nb_epoch', type=int, default=20)
ap.add_argument('-mode', default='train')
ap.add_argument('-weights', default='')

args = vars(ap.parse_args())

DATA_DIR = args['data_dir']
BATCH_SIZE = args['batch_size']
HIDDEN_DIM = args['hidden_dim']
SEQ_LENGTH = args['seq_length']
WEIGHTS = args['weights']

GENERATE_LENGTH = args['generate_length']
LAYER_NUM = args['layer_num']


In [9]:
#print (SEQ_LENGTH)
data = open(args['data_dir'], 'r').read()
chars = list(set(data))
VOCAB_SIZE = len(chars)
print (type(SEQ_LENGTH),type(len(data)))
print (np.zeros(int(len(data)/SEQ_LENGTH),dtype = int))
input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
print(input_sequence[6])

<type 'int'> <type 'int'>
[0 0 0 ... 0 0 0]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
X, y, VOCAB_SIZE, ix_to_char = load_data(DATA_DIR, SEQ_LENGTH)
print('ok')

Data length: 2173397 characters
Vocabulary size: 94 characters
ok


In [0]:
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
  model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


In [14]:
generate_text(model, args['generate_length'], VOCAB_SIZE, ix_to_char)

if not WEIGHTS == '':
  model.load_weights(WEIGHTS)
  nb_epoch = int(WEIGHTS[WEIGHTS.rfind('_') + 1:WEIGHTS.find('.')])
else:
  nb_epoch = 0

  
if args['mode'] == 'train' or WEIGHTS == '':
  while True:
    print('\n\nEpoch: {}\n'.format(nb_epoch))
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, nb_epoch=1)
    nb_epoch += 1
    generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
    if nb_epoch % 10 == 0:
      model.save_weights('checkpoint_layer_{}_hidden_{}_epoch_{}.hdf5'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch))
      
      
elif WEIGHTS == '':
  # Loading the trained weights
  model.load_weights(WEIGHTS)
  generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
  print('\n\n')
else:
  print('\n\nNothing to do!')

LUo,,�222eee2uuss ufuu$$aaaaaaaaYYWWWWEEEmEmEGGGGG!!!!QQ4[[EEEddddddd@@@@EEddddd@@@d@EEddddd@@@@EEdddddd@@@@EEddddd@@@d@EEddddd@@@@EEdddddd@@@@EEddddd@@@d@EEddddd@@@@EEdddddd@@@@EEddddd@@@d@EEddddd@@@@EEdddddd@@@@EEddddd@@@d@EEddddd@@@@EEdddddd@@@@EEddddd@@@d@EEddddd@@@@EEdddddd@@@@EEddddd@@@d@EEddddd@@@@EEdddddd@@@@EEddddd@@@d@EEddddd@@@@EEdddddd@@@@EEddddd@@@d@EEddddd@@@@EEdddddd@@@@EEddddd@@@d@EEddddd@@@@EEdddddd@@@@EEddddd@@@d@EEddddd@@@@EEdddddd@@@@E

Eddddd@@@d@EEddddd@@@@EEdddddd@@@@EEddddd

Epoch: 0



/usr/local/lib/python2.7/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
43467/43467 [==============================] - 110s 3ms/step - loss: 2.5081
9 the seare the sechare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the server

e the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere the stare the servere th

Epoch: 1

Epoch 1/1
30600/43467 [====================>.........] - ETA: 32s - loss: 1.8155

...

he study of 23.40. I thought I believe that the fans that is important to consumers will be able to control the frame of its software them to make a system that are used to represent users to search security essablisment are finding more than the experience." Italy had to make the most of a chance of see himself to be a lifeligning point in midnight on the cap. "We want to do in th

Epoch: 31

Epoch 1/1
 5400/43467 [==>...........................] - ETA: 1:36 - loss: 0.5319

43467/43467 [==============================] - 109s 3ms/step - loss: 0.5799
A problem is not able to take part in the action. "I think we have got to get the ball in our hands and we do everything we can do something with your personal data, like passworns," he said. Studies have shown that consoles had experienced an interesting lise called light more than 120 Borders rely until the seven more than a currently in a 

real test-and partner Martyn Wells told BBC Sport. "We've got the ones that would be the attorney getting a lot of find is in there and a half months for th

Epoch: 32

Epoch 1/1
35000/43467 [=======================>......] - ETA: 21s - loss: 0.5618

43467/43467 [==============================] - 109s 3ms/step - loss: 0.5696
; M Willem, J Worsley, H O'Connor, C Sweeney, D Grewceck, B Drogba (New Zealan) v Readin).

Murray, on the other hopes to lead the work of an added one, if there is a hamstring problem is had so narrow what is in the far corner. I can share so be additional drugs tests in the side." Williams also done alongside a couple of decision." "It is a working toward in ways that I was an absolute 

for them to get a lot of pressure on their computer or going to the point but we now win in the first half.

Epoch: 33

Epoch 1/1
 8000/43467 [====>.........................] - ETA: 1:28 - loss: 0.5173Buffered data was truncated after reaching the output size limit.

In [0]:
from keras.models import load_model

model.save('model.h5')

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 500)         1190000   
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 500)         2002000   
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 94)          47094     
_________________________________________________________________
activation_1 (Activation)    (None, None, 94)          0         
Total params: 3,239,094
Trainable params: 3,239,094
Non-trainable params: 0
_________________________________________________________________


In [0]:
from pickle import dump
from keras.preprocessing.text import Tokenizer

dump(Tokenizer, open('tokenizer.pkl', 'wb'))

##download
files.download('model.h5')
files.download('tokenizer.pkl')